# VacationPy
---

Importing Libraries and Loading the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Additional dependencies
from geoviews import opts

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,8.93,91,100,1.54,US,1724677542
1,1,waitangi,-43.9535,-176.5597,8.79,97,95,0.89,NZ,1724677544
2,2,xining,36.6167,101.7667,16.43,85,68,1.35,CN,1724677545
3,3,al jawf,29.5000,38.7500,39.74,12,2,1.91,SA,1724677546
4,4,port-aux-francais,-49.3500,70.2167,3.06,83,9,16.61,TF,1724677425


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display - Commented out to display the map

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    color='Humidity', 
    frame_width=800, 
    frame_height=600,
    title='City Humidity Scatter Plot'
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity)

In [5]:
#Using geoviews to create a clearer version of the map

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    color='Humidity', 
    geo=True, 
    tiles='OSM',  # OpenStreetMap tiles for the background
    frame_width=800, 
    frame_height=600,
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Defining the criteria
ideal_temp_min_c = 20  # Minimum ideal temperature in Celsius
ideal_temp_max_c = 27  # Maximum ideal temperature in Celsius
ideal_humidity_max = 100  # Maximum ideal humidity percentage
ideal_cloudiness_max = 0  # Maximum ideal cloudiness percentage

# Narrowing down to cities that fit the criteria
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temp_min_c) & 
    (city_data_df['Max Temp'] <= ideal_temp_max_c) & 
    (city_data_df['Humidity'] <= ideal_humidity_max) &
    (city_data_df['Cloudiness'] == ideal_cloudiness_max)
]

# Dropping any rows with null values 
ideal_cities_df = ideal_cities_df.dropna()

# Displaying the sample data
ideal_cities_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,iskateley,67.6803,53.1512,22.94,49,0,6.64,RU,1724677553
43,43,albany,42.6001,-73.9662,20.23,91,0,1.49,US,1724677604
49,49,jamestown,42.0970,-79.2353,20.77,87,0,1.54,US,1724677612
85,85,tolanaro,-25.0319,46.9987,23.31,68,0,9.88,MG,1724677665
93,93,tazacorte,28.6290,-17.9293,24.54,69,0,6.17,ES,1724677676
105,105,port mathurin,-19.6833,63.4167,21.06,59,0,12.57,MU,1724677692
114,114,fort bragg,35.1390,-79.0060,23.97,85,0,1.34,US,1724677704
137,137,port saint john's,-31.6229,29.5448,25.35,48,0,4.35,ZA,1724677737
147,147,emerald,-23.5333,148.1667,20.50,58,0,3.48,AU,1724677751
148,148,bardstown,37.8092,-85.4669,24.68,85,0,1.54,US,1724677752


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Using the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()

# Keeping only the relevant columns (city, country, coordinates, and humidity)
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Adding an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Displaying sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
8,iskateley,RU,67.6803,53.1512,49,
43,albany,US,42.6001,-73.9662,91,
49,jamestown,US,42.0970,-79.2353,87,
85,tolanaro,MG,-25.0319,46.9987,68,
93,tazacorte,ES,28.6290,-17.9293,69,
105,port mathurin,MU,-19.6833,63.4167,59,
114,fort bragg,US,35.1390,-79.0060,85,
137,port saint john's,ZA,-31.6229,29.5448,48,
147,emerald,AU,-23.5333,148.1667,58,
148,bardstown,US,37.8092,-85.4669,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Setting parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Searching for hotels
    "limit": 1,                            # Limiting to the first result
    "apiKey": geoapify_key,                
}

# Printing a message to follow up the hotel search
print("Starting hotel search")

# Iterating through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # getting latitud & longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]

    # Adding the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"

    # Setting base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Making and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Converting the API response to JSON format
    name_address = response.json()

    # Grabbing the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Logging the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Displaying sample data
hotel_df.head(20)

Starting hotel search
iskateley - nearest hotel: Авантаж
albany - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
tolanaro - nearest hotel: Hôtel Mahavokey
tazacorte - nearest hotel: App Leyma
port mathurin - nearest hotel: Escale Vacances
fort bragg - nearest hotel: Airborne Inn Lodging
port saint john's - nearest hotel: Outback Inn
emerald - nearest hotel: Emerald Central Hotel
bardstown - nearest hotel: Talbott Inn
port isabel - nearest hotel: Queen Isabel Inn
tallulah - nearest hotel: Days Inn
hamilton - nearest hotel: North Vista Manor
partenit - nearest hotel: Sunrise
visby - nearest hotel: Hotell Gute
darwin - nearest hotel: Mantra Pantanas Darwin
aripuana - nearest hotel: No hotel found
baise city - nearest hotel: 百色宾馆
east london - nearest hotel: No hotel found
van wert - nearest hotel: Holiday Inn Express & Suites Van Wert
dodge city - nearest hotel: Holiday Inn Express & Suites Dodge City
itapirapua - nearest hotel: No hotel found
podporozh'ye -

,City,Country,Lat,Lng,Humidity,Hotel Name
8,iskateley,RU,67.6803,53.1512,49,Авантаж
43,albany,US,42.6001,-73.9662,91,No hotel found
49,jamestown,US,42.0970,-79.2353,87,DoubleTree Jamestown
85,tolanaro,MG,-25.0319,46.9987,68,Hôtel Mahavokey
93,tazacorte,ES,28.6290,-17.9293,69,App Leyma
105,port mathurin,MU,-19.6833,63.4167,59,Escale Vacances
114,fort bragg,US,35.1390,-79.0060,85,Airborne Inn Lodging
137,port saint john's,ZA,-31.6229,29.5448,48,Outback Inn
147,emerald,AU,-23.5333,148.1667,58,Emerald Central Hotel
148,bardstown,US,37.8092,-85.4669,85,Talbott Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
#%%capture --no-display 'Commented out to display the map'

# Adding 'Hotel Name' to 'hover_cols' and including 'Country'
hover_info = ['City', 'Hotel Name', 'Country']

# Configuring the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    color='Humidity', 
    geo=True, 
    tiles='OSM',  # OpenStreetMap tiles for the background
    frame_width=800, 
    frame_height=600,
    hover_cols=hover_info  #hover information
)

# Displaying the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)